## VGG16 model for training Food101 datasets
- number of parameters : 138,357,544

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms, datasets

from torchinfo import summary

from pathlib import Path

import sys
sys.path.append('..')

from python_scripts import model_builder, engine, data_setup

c:\Users\TimJimTangtong\Miniconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [3]:
food101_transforms_train_2 = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

food101_transforms_test_2 = transforms.Compose([
    transforms.Resize(225),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [4]:
learning_rate_list = [5e-5]
weight_decay_list = [5e-4]
epochs_list = [30]
batch_size_list = [16]

In [5]:
# when using colab: /content/extracted
# when using in window: C:/Users/TimJimTangtong/Deep learning projects/DL_Paper_Replicating/data/extracted
data_dir = Path('C:/Users/TimJimTangtong/Deep learning projects/DL_Paper_Replicating/data/extracted')

train_data_food101 = datasets.Food101(
    root=data_dir,
    split='train',
    transform=food101_transforms_train_2,
    download=True
)
test_data_food101 = datasets.Food101(
    root=data_dir,
    split='test',
    transform=food101_transforms_test_2,
    download=True
)

class_names_food101 = train_data_food101.classes

In [6]:
vgg16_weights = torchvision.models.VGG16_Weights.DEFAULT
def model_generator(weights: torchvision.models):
    model = torchvision.models.vgg16(weights=weights)
    model.classifier[-1] = nn.Linear(
        in_features=4096,
        out_features=len(class_names_food101)
    )
    return model

In [7]:
train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_food101,
    split_size=0.1,
    seed=42
)
test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_food101,
    split_size=0.1,
    seed=42
)

tuning_results = engine.HP_tune_train(
    model_generator=model_generator,
    model_weights=vgg16_weights,
    model_name='VGG16_food101',
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_tensorboard_writer=True,
    device=device,
    gradient_accumulation_num=16
)

[INFO] Splitting dataset of length 75750 into splits of size: 7575 and 68175
[INFO] Splitting dataset of length 25250 into splits of size: 2525 and 22725
[INFO] Creating SummaryWriter saving to ../runs\2023-03-17\VGG16_food101_test\VGG16_food101\LR_0.0005_WD_0.001_EP_15_BS_16_GA_16


  7%|▋         | 1/15 [02:19<32:32, 139.47s/it]

Epoch: 0 | Train_loss: 4.6286, Train_acc: 0.0092 | Test_loss: 4.6182, Test_acc: 0.0095


 13%|█▎        | 2/15 [04:34<29:39, 136.85s/it]

Epoch: 1 | Train_loss: 4.6157, Train_acc: 0.0141 | Test_loss: 4.6205, Test_acc: 0.0059


...

Epoch: 12 | Train_loss: 3.3273, Train_acc: 0.2247 | Test_loss: 2.3799, Test_acc: 0.4148


 93%|█████████▎| 14/15 [31:39<02:15, 135.78s/it]

Epoch: 13 | Train_loss: 3.2944, Train_acc: 0.2258 | Test_loss: 2.2687, Test_acc: 0.4416


100%|██████████| 15/15 [33:54<00:00, 135.66s/it]

Epoch: 14 | Train_loss: 3.2207, Train_acc: 0.2427 | Test_loss: 2.2438, Test_acc: 0.4464


In [8]:
import pandas as pd
pd.DataFrame(tuning_results)

,learning_rate,weight_decay,epochs,batch_size,train_loss,test_loss,train_acc,test_acc
0,0.00050,0.0010,15,16,4.457926,4.3640804,0.025975738,0.025316456
1,0.00050,0.0005,15,16,4.557368,4.5417724,0.020042194,0.01938291
2,0.00050,0.0001,15,16,4.409796,4.293235,0.03388713,0.03164557
3,0.00010,0.0010,15,16,2.2622802,1.7338767,0.4391011,0.55452776
4,0.00010,0.0005,15,16,2.161479,1.6662295,0.4647001,0.5734238
5,0.00010,0.0001,15,16,2.4460797,1.780208,0.40082505,0.5307936
6,0.00005,0.0010,15,16,2.3513014,1.647894,0.41568717,0.5604613
7,0.00005,0.0005,15,16,2.347944,1.6606896,0.42544457,0.5528542
8,0.00005,0.0001,15,16,2.3535194,1.6510093,0.41404837,0.5528542
9,0.00001,0.0010,15,16,3.1870146,2.2709146,0.24154235,0.43606988


In [9]:
vgg16_bn_weights = torchvision.models.VGG16_BN_Weights
def vgg16_bn_model_generator(weights: torchvision.models):
    model = torchvision.models.vgg16_bn(weights=weights)
    model.classifier[-1] = nn.Linear(
        in_features=4096,
        out_features=len(class_names_food101)
    )
    return model

In [10]:
train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_food101,
    split_size=0.1,
    seed=42
)
test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_food101,
    split_size=0.1,
    seed=42
)

tuning_results = engine.HP_tune_train(
    model_generator=vgg16_bn_model_generator,
    model_weights=vgg16_bn_weights,
    model_name='VGG16_bn_food101',
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_tensorboard_writer=True,
    device=device,
    gradient_accumulation_num=1
)

[INFO] Splitting dataset of length 75750 into splits of size: 7575 and 68175
[INFO] Splitting dataset of length 25250 into splits of size: 2525 and 22725


c:\Users\TimJimTangtong\Miniconda3\envs\torch-gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[INFO] Creating SummaryWriter saving to ../runs\2023-03-18\VGG16_bn_food101_test\VGG16_bn_food101\LR_0.0005_WD_0.001_EP_15_BS_16_GA_1


  7%|▋         | 1/15 [02:53<40:23, 173.13s/it]

Epoch: 0 | Train_loss: 4.6457, Train_acc: 0.0104 | Test_loss: 4.6405, Test_acc: 0.0067


...

Epoch: 12 | Train_loss: 2.3910, Train_acc: 0.4123 | Test_loss: 1.4538, Test_acc: 0.5982


 93%|█████████▎| 14/15 [40:30<02:53, 173.74s/it]

Epoch: 13 | Train_loss: 2.3807, Train_acc: 0.4088 | Test_loss: 1.4413, Test_acc: 0.6068


100%|██████████| 15/15 [43:24<00:00, 173.65s/it]

Epoch: 14 | Train_loss: 2.3231, Train_acc: 0.4322 | Test_loss: 1.4058, Test_acc: 0.6111


In [11]:
import pandas as pd
pd.DataFrame(tuning_results)

,learning_rate,weight_decay,epochs,batch_size,train_loss,test_loss,train_acc,test_acc
0,0.00050,0.0010,15,16,4.5276074,4.453647,0.023206752,0.01898734
1,0.00050,0.0005,15,16,4.548298,4.491308,0.022453286,0.012262658
2,0.00050,0.0001,15,16,4.615389,4.656549,0.015559072,0.011867088
3,0.00010,0.0010,15,16,2.2501433,1.7973984,0.44648507,0.55452776
4,0.00010,0.0005,15,16,2.1077619,1.7670223,0.47453284,0.5587877
5,0.00010,0.0001,15,16,2.0238347,1.5908701,0.49853075,0.5938717
6,0.00005,0.0010,15,16,1.8958741,1.4589834,0.5261453,0.61979675
7,0.00005,0.0005,15,16,1.8218819,1.4074081,0.5400655,0.630964
8,0.00005,0.0001,15,16,1.778301,1.499908,0.545547,0.6182145
9,0.00001,0.0010,15,16,2.3282585,1.4506983,0.4256141,0.59082884


In [6]:
vgg16_weights = torchvision.models.VGG16_Weights.DEFAULT
vgg16_model = torchvision.models.vgg16(weights=vgg16_weights)
vgg16_model.classifier[-1] = torch.nn.Linear(
    in_features=4096,
    out_features=len(class_names_food101)
)

In [7]:
train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_food101,
    split_size=1,
    seed=42
)
test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_food101,
    split_size=1,
    seed=42
)

tuning_results = engine.HP_tune_train(
    model=vgg16_model,
    model_generator=None,
    model_weights=vgg16_weights,
    model_name='VGG16_food101',
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_tensorboard_writer=True,
    device=device,
    gradient_accumulation_num=16
)

[INFO] Splitting dataset of length 75750 into splits of size: 75750 and 0
[INFO] Splitting dataset of length 25250 into splits of size: 25250 and 0
[INFO] Creating SummaryWriter saving to ../runs\2023-03-18\VGG16_food101_test\VGG16_food101\LR_5e-05_WD_0.0005_EP_30_BS_16_GA_16


  3%|▎         | 1/30 [23:10<11:11:51, 1390.04s/it]

Epoch: 0 | Train_loss: 3.4985, Train_acc: 0.1967 | Test_loss: 1.7645, Test_acc: 0.5428


  7%|▋         | 2/30 [46:17<10:47:59, 1388.56s/it]

Epoch: 1 | Train_loss: 2.6557, Train_acc: 0.3572 | Test_loss: 1.4274, Test_acc: 0.6217
Epoch: 2 | Train_loss: 2.3919, Train_acc: 0.4153 | Test_loss: 1.2596, Test_acc: 0.6581


...

Epoch: 27 | Train_loss: 1.3368, Train_acc: 0.6558 | Test_loss: 0.7019, Test_acc: 0.8086


 97%|█████████▋| 29/30 [11:23:52<23:45, 1425.18s/it]

Epoch: 28 | Train_loss: 1.3259, Train_acc: 0.6590 | Test_loss: 0.7247, Test_acc: 0.8013


100%|██████████| 30/30 [11:47:14<00:00, 1414.47s/it]

Epoch: 29 | Train_loss: 1.3117, Train_acc: 0.6603 | Test_loss: 0.6924, Test_acc: 0.8105


In [8]:
import pandas as pd
pd.DataFrame(tuning_results)

,learning_rate,weight_decay,epochs,batch_size,train_loss,test_loss,train_acc,test_acc
0,0.00005,0.0005,30,16,1.3117051,0.69244385,0.6603485,0.8104813


In [6]:
vgg16_bn_weights = torchvision.models.VGG16_BN_Weights.DEFAULT
vgg16_bn_model = torchvision.models.vgg16_bn(weights=vgg16_bn_weights)
vgg16_bn_model.classifier[-1] = torch.nn.Linear(
    in_features=4096,
    out_features=len(class_names_food101)
)

In [7]:
train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_food101,
    split_size=1,
    seed=42
)
test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_food101,
    split_size=1,
    seed=42
)

tuning_results = engine.HP_tune_train(
    model=vgg16_bn_model,
    model_generator=None,
    model_weights=vgg16_bn_weights,
    model_name='VGG16_bn_food101',
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_tensorboard_writer=True,
    device=device,
    gradient_accumulation_num=1
)

[INFO] Splitting dataset of length 75750 into splits of size: 75750 and 0
[INFO] Splitting dataset of length 25250 into splits of size: 25250 and 0
[INFO] Creating SummaryWriter saving to ../runs\2023-03-19\VGG16_bn_food101_test\VGG16_bn_food101\LR_5e-05_WD_0.0005_EP_30_BS_16_GA_1


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 0 | Train_loss: 2.9199, Train_acc: 0.3062 | Test_loss: 1.3659, Test_acc: 0.6346


  7%|▋         | 2/30 [58:37<13:41:00, 1759.30s/it]

Epoch: 1 | Train_loss: 2.2767, Train_acc: 0.4412 | Test_loss: 1.1121, Test_acc: 0.6992


...

Epoch: 27 | Train_loss: 1.4250, Train_acc: 0.6440 | Test_loss: 0.7343, Test_acc: 0.8019


 97%|█████████▋| 29/30 [14:13:15<29:23, 1763.84s/it]

Epoch: 28 | Train_loss: 1.4268, Train_acc: 0.6422 | Test_loss: 0.7375, Test_acc: 0.8038


100%|██████████| 30/30 [14:42:40<00:00, 1765.35s/it]

Epoch: 29 | Train_loss: 1.4117, Train_acc: 0.6433 | Test_loss: 0.7103, Test_acc: 0.8063


In [8]:
import pandas as pd
pd.DataFrame(tuning_results)

,learning_rate,weight_decay,epochs,batch_size,train_loss,test_loss,train_acc,test_acc
0,0.00005,0.0005,30,16,1.4116825,0.71032387,0.64334744,0.8063252
